In [1]:
# 1. Install and import libraries

!pip install requests pandas lxml

import requests, json, base64, hashlib, secrets, urllib.parse
import pandas as pd
import datetime

In [ ]:
# 2. Yahoo app credentials

CLIENT_ID = "ENTER YOUR CLIENT_ID HERE"
REDIRECT_URI = "https://pythonanywhere.com"
BASE_URL = "https://api.login.yahoo.com/"
SCOPE = "fspt-r"

def create_pkce_pair():
    verifier = secrets.token_urlsafe(64)
    challenge = base64.urlsafe_b64encode(
        hashlib.sha256(verifier.encode("utf-8")).digest()
    ).rstrip(b"=").decode("ascii")
    return verifier, challenge

code_verifier, code_challenge = create_pkce_pair()

params = {
    "client_id": CLIENT_ID,
    "redirect_uri": REDIRECT_URI,
    "response_type": "code",
    "scope": SCOPE,
    "code_challenge": code_challenge,
    "code_challenge_method": "S256",
}

auth_url = BASE_URL + "oauth2/request_auth?" + urllib.parse.urlencode(params)
print("OPEN THIS URL IN YOUR BROWSER AND AUTHORIZE ACCESS:\n")
print(auth_url)


In [ ]:
# 3. Exchange code for access token

code = input("Paste the 'code' from the URL: ").strip()

token_url = BASE_URL + "oauth2/get_token"

headers = {"Content-Type": "application/x-www-form-urlencoded"}

data = {
    "grant_type": "authorization_code",
    "client_id": CLIENT_ID,
    "redirect_uri": REDIRECT_URI,
    "code": code,
    "code_verifier": code_verifier,
}

response = requests.post(token_url, headers=headers, data=data)
print("Status:", response.status_code)
print("Raw response:", response.text)

tokens = response.json()
access_token = tokens["access_token"]
refresh_token = tokens.get("refresh_token")
print("\nAccess Token OK.")

In [ ]:
# 4. Auth Get Helper function

def yahoo_get(endpoint: str):
    """
    Wrapper that returns parsed JSON for any Yahoo Fantasy endpoint.
    """
    url = f"https://fantasysports.yahooapis.com/fantasy/v2/{endpoint}?format=json"
    headers = {"Authorization": f"Bearer {access_token}"}
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.json()

In [ ]:
# 5. Retreive Current NBA GAME_KEY

def collect_game_keys(obj, collected=None):
    """Recursively collect all 'game_key' values in a nested Yahoo JSON object."""
    if collected is None:
        collected = []

    if isinstance(obj, dict):
        if "game_key" in obj:
            collected.append(obj["game_key"])
        for v in obj.values():
            collect_game_keys(v, collected)
    elif isinstance(obj, list):
        for item in obj:
            collect_game_keys(item, collected)

    return collected

def get_latest_nba_game_key():
    j = yahoo_get("games;game_codes=nba")
    games = j.get("fantasy_content", {}).get("games", {})
    all_keys = collect_game_keys(games, [])

    if not all_keys:
        print("No game_key values found. Raw 'games' structure:")
        print(json.dumps(games, indent=2)[:2000])
        raise RuntimeError("Could not find any NBA game_key values.")

    # Yahoo uses numeric-ish game keys per season; pick the latest (max)
    # Filter to those that look like integers, just in case.
    numeric_keys = []
    for k in all_keys:
        try:
            numeric_keys.append(int(str(k)))
        except ValueError:
            pass

    if not numeric_keys:
        print("Non-numeric game_keys:", all_keys)
        raise RuntimeError("Could not interpret NBA game_keys as integers.")

    latest = max(numeric_keys)
    return str(latest)

GAME_KEY = get_latest_nba_game_key()
print("NBA Game Key (latest season):", GAME_KEY)

In [ ]:
# 6. League Info & Team IDs ***ACTION REQUIRED***

LEAGUE_ID = ""     # <-- Your league
MY_TEAM_ID = ""      # <-- Your team
OPP_TEAM_ID = ""      # <-- Your opponent. MUST CHANGE EACH WEEK.

LEAGUE_KEY = f"{GAME_KEY}.l.{LEAGUE_ID}"

In [ ]:
# 7. Roster Extraction

def _extract_nested_dict(obj, keys):
    """
    Helper to navigate Yahoo's sometimes inconsistent nested list/dict structures.
    It attempts to drill down through keys, unwrapping single-item lists if found.
    """
    current_obj = obj
    for key in keys:
        if not isinstance(current_obj, dict):
            return {} # Path broken, return empty dict

        next_level = current_obj.get(key, {})

        # If it's a list, try to unwrap it if it contains a single item
        if isinstance(next_level, list) and len(next_level) > 0:
            # Check if the first item is itself a list (e.g. [[dict]])
            if isinstance(next_level[0], list) and len(next_level[0]) > 0:
                current_obj = next_level[0][0] # Get dict from [[dict]]
            else:
                current_obj = next_level[0] # Get dict from [dict]
        else:
            current_obj = next_level # Not a list, just move to next dict

    # Ensure the final result is a dict (or we return an empty dict)
    return current_obj if isinstance(current_obj, dict) else {}

def get_team_roster(team_id):
    team_key = f"{LEAGUE_KEY}.t.{team_id}"
    # Corrected: Removed '?format=json' as yahoo_get already adds it
    data = yahoo_get(f"team/{team_key}/roster/players")

    rows = []
    try:
        fantasy_content = data.get('fantasy_content', {})
        team_list = fantasy_content.get('team', [])

        if len(team_list) > 1 and isinstance(team_list[1], dict):
            roster_info = team_list[1].get('roster', {})
            players_container = roster_info.get('0', {})
            roster_players_dict = players_container.get('players', {})

            # Filter out non-player entries like 'count'
            roster_players = [v for k, v in roster_players_dict.items() if isinstance(v, dict)]
        else:
            print(f"Could not find roster data for team {team_id} in expected location.")
            return pd.DataFrame(columns=["player_id", "name", "pos", "nba_team"])

        for player_item_wrapper in roster_players:
            # player_item_wrapper is like {'player': [ [player_attributes_list], selected_position_dict, is_editable_dict ]}
            player_details_raw = player_item_wrapper.get('player', [])

            pid, name, pos, nba_team = None, None, None, None

            if player_details_raw and isinstance(player_details_raw[0], list):
                # The actual player attributes are in the first element of player_details_raw
                player_attributes_list = player_details_raw[0]

                for item_dict in player_attributes_list:
                    if isinstance(item_dict, dict):
                        if 'player_id' in item_dict:
                            pid = item_dict['player_id']
                        if 'name' in item_dict and 'full' in item_dict['name']:
                            name = item_dict['name']['full']
                        if 'primary_position' in item_dict:
                            pos = item_dict['primary_position']
                        if 'editorial_team_full_name' in item_dict:
                            nba_team = item_dict['editorial_team_full_name']

            if pid is not None:
                rows.append({"player_id": pid, "name": name, "pos": pos, "nba_team": nba_team})

    except (KeyError, IndexError, ValueError) as e:
        print(f"Error parsing Yahoo API response for team roster: {e}")
        print("Please check the structure of the API response for team roster data.")
        print(f"Partial API response for debugging (first 1000 chars): {json.dumps(data, indent=2)[:1000]}...")
        return pd.DataFrame(columns=["player_id", "name", "pos", "nba_team"])

    return pd.DataFrame(rows).drop_duplicates("player_id")


my_team = get_team_roster(MY_TEAM_ID)
opp_team = get_team_roster(OPP_TEAM_ID)

my_team, opp_team

In [ ]:
# 8. Define the scoring week: Monday through Sunday
today = datetime.date.today()
monday = today - datetime.timedelta(days=today.weekday())
week_dates = [monday + datetime.timedelta(days=i) for i in range(7)]

print("Week dates:", week_dates)
print("Today:", today)

In [ ]:
#. 9 Helper to walk nested Yahoo JSON

def walk_for_key(obj, target_key):
    """
    Recursively search nested dict/list for dicts that contain target_key.
    Yield each dict that directly contains that key.
    """
    if isinstance(obj, dict):
        if target_key in obj:
            yield obj
        for v in obj.values():
            yield from walk_for_key(v, target_key)
    elif isinstance(obj, list):
        for v in obj:
            yield from walk_for_key(v, target_key)

In [ ]:
# Stat modifiers

def load_stat_modifiers():
    """
    Load stat modifiers for your league from league settings.

    Returns:
        dict: { stat_id (str) -> weight (float) }
    """
    settings = yahoo_get(f"league/{LEAGUE_KEY}/settings")
    fc = settings.get("fantasy_content", {})

    modifiers = {}

    # Look for the 'stat_modifiers' block in the league settings
    for holder in walk_for_key(fc, "stat_modifiers"):
        sm = holder.get("stat_modifiers", {})
        stats_list = sm.get("stats", [])
        if not isinstance(stats_list, list):
            continue

        for item in stats_list:
            stat_obj = item.get("stat", {})
            sid = str(stat_obj.get("stat_id"))
            val = stat_obj.get("value")

            try:
                w = float(val)
            except (TypeError, ValueError):
                continue

            modifiers[sid] = w

        # We assume the first stat_modifiers we find is the right one
        break

    print("Loaded stat modifiers (first 10):")
    print(dict(list(modifiers.items())[:10]))
    return modifiers


STAT_WEIGHTS = load_stat_modifiers()

In [ ]:
def get_player_season_avg(player_id: str) -> float:
    """
    Compute season-average fantasy points for a player using:
      player/{GAME_KEY}.p.{player_id}/stats;type=season

    We:
      - Pull all season stats for the player
      - Use league STAT_WEIGHTS to compute total fantasy points
      - Divide by Games Played (stat_id == "0") to get per-game average
    """
    player_key = f"{GAME_KEY}.p.{player_id}"
    endpoint = f"player/{player_key}/stats;type=season"
    data = yahoo_get(endpoint)

    fc = data.get("fantasy_content", {})

    # Find the player_stats->stats block
    stats_list = None
    for holder in walk_for_key(fc, "player_stats"):
        ps = holder.get("player_stats", {})
        stats_list = ps.get("stats")
        if stats_list:
            break

    if not stats_list:
        return 0.0

    total_fp = 0.0
    games_played = None

    for item in stats_list:
        stat = item.get("stat", {})
        sid = str(stat.get("stat_id"))
        val_str = stat.get("value")

        if val_str in (None, "-", ""):
            continue

        try:
            val = float(val_str)
        except (TypeError, ValueError):
            continue

        # stat_id 0 is Games Played
        if sid == "0":
            games_played = val

        # Apply fantasy scoring weight if present
        w = STAT_WEIGHTS.get(sid)
        if w is not None and w != 0:
            total_fp += val * w

    # Prefer per-game average
    if games_played and games_played > 0:
        return total_fp / games_played

    # Fallback (unlikely): return total season points if no GP found
    return total_fp

In [ ]:
my_team["avg_fp"] = my_team["player_id"].astype(str).apply(get_player_season_avg)
opp_team["avg_fp"] = opp_team["player_id"].astype(str).apply(get_player_season_avg)

print("My team with season-average FP:")
display(my_team)

print("\nOpponent team with season-average FP:")
display(opp_team)

In [ ]:
# Function to pull current matchup score

def get_current_matchup_scores(my_team_id: int, opp_team_id: int):
    """
    Fetch the current week's scoreboard for your league and return
    the official Yahoo fantasy points (and projected points if available)
    for your team and your opponent.

    Returns a dict like:
      {
        my_team_id:  {"points": 732.1, "projected": 1090.4},
        opp_team_id: {"points": 637.0, "projected": 1012.3}
      }
    """
    # Default scoreboard = current week
    data = yahoo_get(f"league/{LEAGUE_KEY}/scoreboard")
    fc = data.get("fantasy_content", {})

    scores = {}

    # Walk through all 'team' blocks in the scoreboard
    for holder in walk_for_key(fc, "team"):
        team_obj = holder.get("team")
        if not isinstance(team_obj, list) or not team_obj:
            continue

        meta_block = team_obj[0]   # list of small dicts: team_key, team_id, name, etc.
        team_id = None

        # Find team_id inside the meta block
        if isinstance(meta_block, list):
            for item in meta_block:
                if isinstance(item, dict) and "team_id" in item:
                    team_id = item["team_id"]
                    break

        if team_id is None:
            continue

        try:
            team_id_int = int(team_id)
        except ValueError:
            continue

        # Only care about your team and your opponent
        if team_id_int not in (my_team_id, opp_team_id):
            continue

        team_points = None
        team_proj = None

        # Remaining elements may contain team_points, team_projected_points, etc.
        for item in team_obj[1:]:
            if not isinstance(item, dict):
                continue

            if "team_points" in item:
                tp = item["team_points"]
                try:
                    team_points = float(tp.get("total", 0) or 0)
                except (TypeError, ValueError):
                    team_points = 0.0

            if "team_projected_points" in item:
                tpp = item["team_projected_points"]
                try:
                    team_proj = float(tpp.get("total", 0) or 0)
                except (TypeError, ValueError):
                    team_proj = 0.0

        scores[team_id_int] = {
            "points": team_points,
            "projected": team_proj
        }

    return scores

In [ ]:
# Call for current week

scores = get_current_matchup_scores(int(MY_TEAM_ID), int(OPP_TEAM_ID))
print("Current matchup scores (Yahoo official):")
print(scores)

my_current_pts = scores.get(int(MY_TEAM_ID), {}).get("points")
opp_current_pts = scores.get(int(OPP_TEAM_ID), {}).get("points")

print(f"\nMy team (ID {MY_TEAM_ID}) current points:   {my_current_pts}")
print(f"Opp team (ID {OPP_TEAM_ID}) current points: {opp_current_pts}")

In [ ]:
# NBA team name mapping

TEAM_NAME_TO_ABBR = {
    "Atlanta Hawks": "ATL",
    "Boston Celtics": "BOS",
    "Brooklyn Nets": "BKN",
    "Charlotte Hornets": "CHA",
    "Chicago Bulls": "CHI",
    "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL",
    "Denver Nuggets": "DEN",
    "Detroit Pistons": "DET",
    "Golden State Warriors": "GS",
    "Houston Rockets": "HOU",
    "Indiana Pacers": "IND",
    "Los Angeles Clippers": "LAC",
    "Los Angeles Lakers": "LAL",
    "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA",
    "Milwaukee Bucks": "MIL",
    "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NO",
    "New York Knicks": "NY",
    "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL",
    "Philadelphia 76ers": "PHI",
    "Phoenix Suns": "PHX",
    "Portland Trail Blazers": "POR",
    "Sacramento Kings": "SAC",
    "San Antonio Spurs": "SA",
    "Toronto Raptors": "TOR",
    "Utah Jazz": "UTAH",
    "Washington Wizards": "WSH",
}

In [ ]:
# Add mapped abbr to existing tables

my_team["team_abbr"]  = my_team["nba_team"].map(TEAM_NAME_TO_ABBR)
opp_team["team_abbr"] = opp_team["nba_team"].map(TEAM_NAME_TO_ABBR)

In [ ]:
# NBA Schedule API

from datetime import timedelta

def get_nba_schedule_for_fantasy_week():
    rows = []
    # Use the pre-calculated week_dates for the current fantasy week
    for day in week_dates:
        url = f"https://site.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard?dates={day.strftime('%Y%m%d')}"
        data = requests.get(url).json()

        for g in data.get("events", []):
            comps = g["competitions"][0]["competitors"]

            home = next(t for t in comps if t["homeAway"] == "home")
            away = next(t for t in comps if t["homeAway"] == "away")

            rows.append({
                "date": day,
                "team": home["team"]["abbreviation"],
                "opponent": away["team"]["abbreviation"],
                "home_away": "Home"
            })
            rows.append({
                "date": day,
                "team": away["team"]["abbreviation"],
                "opponent": home["team"]["abbreviation"],
                "home_away": "Away"
            })

    return pd.DataFrame(rows)

schedule_df = get_nba_schedule_for_fantasy_week()
schedule_df.head()

In [ ]:
# Create exports directory and save all dataframes to CSV
import os

EXPORT_DIR = "exports"
os.makedirs(EXPORT_DIR, exist_ok=True)

# Save current score to CSV
current_score_df = pd.DataFrame.from_dict(scores, orient='index')
current_score_df.index.name = 'team_id'
current_score_df.to_csv(os.path.join(EXPORT_DIR, "current_score.csv"))
print(f"Saved {os.path.join(EXPORT_DIR, 'current_score.csv')}")

# Save my_team to CSV
my_team.to_csv(os.path.join(EXPORT_DIR, "my_team.csv"), index=False)
print(f"Saved {os.path.join(EXPORT_DIR, 'my_team.csv')}")

# Save opp_team to CSV
opp_team.to_csv(os.path.join(EXPORT_DIR, "opp_team.csv"), index=False)
print(f"Saved {os.path.join(EXPORT_DIR, 'opp_team.csv')}")

# Save nba_schedule_next_7_days to CSV
schedule_df.to_csv(os.path.join(EXPORT_DIR, "nba_schedule_next_7_days.csv"), index=False)
print(f"Saved {os.path.join(EXPORT_DIR, 'nba_schedule_next_7_days.csv')}")

# Now download the created files
from google.colab import files

files_to_download = [
    "exports/current_score.csv",
    "exports/my_team.csv",
    "exports/opp_team.csv",
    "exports/nba_schedule_next_7_days.csv"
]

for f in files_to_download:
    try:
        files.download(f)
        print(f"Downloaded: {f}")
    except Exception as e:
        print(f"Error downloading {f}: {e}")